<a href="https://colab.research.google.com/github/RyuichiSaito1/covid19-twitter-usa-restoring/blob/main/sentiment_classifier_gpt_3_5_turbo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade openai
!pip install --upgrade gspread

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

Mounted at /content/drive


In [ ]:
import os

# https://community.openai.com/t/google-colab-fine-tuning-error/5917


# New York City

In [ ]:
import gspread
import pandas as pd

# Input Files
tsv_files=os.listdir('/content/drive/My Drive/covid-twitter-usa-normal/data/collection_data/new_york_city/3/1/')
# tsv_files=os.listdir('/content/drive/My Drive/covid-twitter-usa-normal/data/collection_data/new_york_city/1/work/')

# Sort the file names
tsv_files_sorted = sorted(tsv_files)

# Output File
url = "https://docs.google.com/spreadsheets/d/1_A7eljHvSSeqpDUEDrMZFU44-durXx1lXoK8k4mAjh0/edit#gid=314682145"
sheet_name = '3'

In [ ]:
import time
from openai import OpenAI

client = OpenAI()

number = 0
limit_control_no = 0

workbook = gc.open_by_url(url)
worksheet = workbook.worksheet(sheet_name)
header = ['file name', 'tweets', 'positive', 'neutral', 'negative', 'score']
worksheet.append_row(header)

# Function to classify sentiment using OpenAI API
def classify_sentiment(text):
    response = client.chat.completions.create(
        model="ft:gpt-3.5-turbo-1106:university-of-tsukuba::8RcZEQZm",
        messages=[
            {"role": "system", "content": "You are a sentiment classifier of tweets from citizens of a large US city collected during the COVID-19 pandemic. Classify into three values: 0 for positive, 1 for neutral, and 2 for negative. Positive sentiment includes: admire, amazing, assure, celebration, charm, eager, enthusiastic, excellent, fancy, fantastic, frolic, graceful, happy, joy, luck, majesty, mercy, nice, patience, perfect, proud, rejoice, relief, respect, satisfactorily, sensational, super, terrific, thank, vivid, wise, wonderful, zest, expectations and etc. Negative sentiment includes: abominable, anger, anxious, bad, catastrophe, cheap, complaint, condescending, deceit, defective, disappointment, embarrass, fake, fear, filthy, fool, guilt, hate, idiot, inflict, lazy, miserable, mourn, nervous, objection, pest, plot, reject, scream, silly, terrible, unfriendly, vile, wicked, and etc. Neutral sentiment: neither positive or negative, such as text without sentiment, stating a fact, question, news article, advertisement, solicitation, request, quote, unintelligible text, and etc. When the sentiment is mixed, such as both joy and sadness, use your judgment to choose the stronger emotion."},
            {"role": "user", "content": text}
        ],
        timeout=60  # Timeout set to 60 seconds
    )

    print("Prediction: " + response.choices[0].message.content)
    print("")
    try:
        return int(response.choices[0].message.content)
    except ValueError:
        print(f"Error: Unable to convert '{response.choices[0].message.content}' to int. Skipping this prediction.")
        return None

for h in range(len(tsv_files)):

    print(str(tsv_files[h]))

    columns = ['id', 'author_id', 'tweet', 'index']
    df_output = pd.DataFrame(columns=columns)

    sum_num = 0
    pos_num = 0
    neu_num = 0
    neg_num = 0
    other_num = 0
    score = 0
    number = 0
    limit_control_no = 0
    sentiment_score = 0
    sentiment_index = 0
    tweets_number = 0

    # ☆
    df_input = pd.read_table('/content/drive/My Drive/covid-twitter-usa-normal/data/collection_data/new_york_city/3/1/' + str(tsv_files[h]), names=('author_id', 'time', 'id', 'text', 'place'))
    print("The number of line:", len(df_input))
    # Duplicate Text Column
    df_input = df_input.drop_duplicates(subset='text')
    print("The number of line after duplication:", len(df_input))
    # Rate Limit Counterplan
    print('Wait for 3 seconds')
    time.sleep(3)

    for index, item in df_input.iterrows():

        print(index)
        text = item['text']
        print(text)

        time.sleep(0.5)

        max_retries = 3
        retries = 0

        while retries < max_retries:
            try:
                predicted_label = classify_sentiment(text)
                break  # If successful, exit the loop
            except Exception as e:
                retries += 1
                print(f"Error: {str(e)}. Retrying {retries}/{max_retries}...")  # Fix: Print retry information
                if retries == max_retries:
                    # Log the error and raise an exception to stop processing
                    print("Max retries exceeded. Logging the error and stopping the processing.")
                    raise
                time.sleep(5)

        number += 1
        limit_control_no += 1

        if predicted_label == 0:
            pos_num += 1
            score = 0
            sum_num += -1
        elif predicted_label == 1:
            neu_num += 1
            score = 1
            sum_num += 0
        elif predicted_label == 2:
            neg_num += 1
            score = 2
            sum_num += 1
        else:
            other_num += 1
            score = 4

        # Rate Limit Counterplan
        if limit_control_no == 200:
            limit_control_no = 0
            print('Wait for 10 seconds')
            time.sleep(10)

        append_list = [str(item['id']), str(item['author_id']), str(item['text']), str(score)]
        df_next = pd.DataFrame([append_list], columns=columns)
        df_output = pd.concat([df_output, df_next], ignore_index=True)

    sentiment_score = sum_num / number
    sentiment_index = 'Sentiment Index: ' + str(sentiment_score)
    tweets_number = 'Tweet_Number: ' + str(number)
    positive_number = str(pos_num)
    neutral_number = str(neu_num)
    negative_number = str(neg_num)
    file_name = 'File_Name: ' + str(tsv_files[h])
    print(file_name + ' ' + tweets_number + ' ' + str(sentiment_index))
    result_set = [str(tsv_files[h]), str(number), positive_number, neutral_number, negative_number, str(sentiment_score)]
    worksheet.append_row(result_set)

    # ☆
    directory = '/content/drive/My Drive/covid-twitter-usa-normal/result/gpt-3.5-turbo/new_york_city/3/' + str(tsv_files[h])
    df_output.to_csv(directory, sep='\t', index=False, header=False)


# Los Angeles

In [ ]:
import gspread
import pandas as pd

# Input Files
tsv_files=os.listdir('/content/drive/My Drive/covid-twitter-usa-normal/data/collection_data/los_angeles/3/1/')
# tsv_files=os.listdir('/content/drive/My Drive/covid-twitter-usa-normal/data/collection_data/new_york_city/1/work/')

# Sort the file names
tsv_files_sorted = sorted(tsv_files)

# Output File
url = "https://docs.google.com/spreadsheets/d/1SjTCyR9rMUkPfuJBa1ZR6YBvJl3masUiQU0GJ7JHCNg/edit#gid=314682145"
sheet_name = '3'

In [ ]:
import time
from openai import OpenAI

client = OpenAI()

number = 0
limit_control_no = 0

workbook = gc.open_by_url(url)
worksheet = workbook.worksheet(sheet_name)
header = ['file name', 'tweets', 'positive', 'neutral', 'negative', 'score']
worksheet.append_row(header)

# Function to classify sentiment using OpenAI API
def classify_sentiment(text):
    response = client.chat.completions.create(
        model="ft:gpt-3.5-turbo-1106:university-of-tsukuba::8RcZEQZm",
        messages=[
            {"role": "system", "content": "You are a sentiment classifier of tweets from citizens of a large US city collected during the COVID-19 pandemic. Classify into three values: 0 for positive, 1 for neutral, and 2 for negative. Positive sentiment includes: admire, amazing, assure, celebration, charm, eager, enthusiastic, excellent, fancy, fantastic, frolic, graceful, happy, joy, luck, majesty, mercy, nice, patience, perfect, proud, rejoice, relief, respect, satisfactorily, sensational, super, terrific, thank, vivid, wise, wonderful, zest, expectations and etc. Negative sentiment includes: abominable, anger, anxious, bad, catastrophe, cheap, complaint, condescending, deceit, defective, disappointment, embarrass, fake, fear, filthy, fool, guilt, hate, idiot, inflict, lazy, miserable, mourn, nervous, objection, pest, plot, reject, scream, silly, terrible, unfriendly, vile, wicked, and etc. Neutral sentiment: neither positive or negative, such as text without sentiment, stating a fact, question, news article, advertisement, solicitation, request, quote, unintelligible text, and etc. When the sentiment is mixed, such as both joy and sadness, use your judgment to choose the stronger emotion."},
            {"role": "user", "content": text}
        ],
        timeout=60  # Timeout set to 60 seconds
    )

    print("Prediction: " + response.choices[0].message.content)
    print("")
    try:
        return int(response.choices[0].message.content)
    except ValueError:
        print(f"Error: Unable to convert '{response.choices[0].message.content}' to int. Skipping this prediction.")
        return None

for h in range(len(tsv_files)):

    print(str(tsv_files[h]))

    columns = ['id', 'author_id', 'tweet', 'index']
    df_output = pd.DataFrame(columns=columns)

    sum_num = 0
    pos_num = 0
    neu_num = 0
    neg_num = 0
    other_num = 0
    score = 0
    number = 0
    limit_control_no = 0
    sentiment_score = 0
    sentiment_index = 0
    tweets_number = 0

    # ☆
    df_input = pd.read_table('/content/drive/My Drive/covid-twitter-usa-normal/data/collection_data/los_angeles/3/1/' + str(tsv_files[h]), names=('author_id', 'time', 'id', 'text', 'place'))
    print("The number of line:", len(df_input))
    # Duplicate Text Column
    df_input = df_input.drop_duplicates(subset='text')
    print("The number of line after duplication:", len(df_input))
    # Rate Limit Counterplan
    print('Wait for 3 seconds')
    time.sleep(3)

    for index, item in df_input.iterrows():

        print(index)
        text = item['text']
        print(text)

        time.sleep(0.5)

        max_retries = 3
        retries = 0

        while retries < max_retries:
            try:
                predicted_label = classify_sentiment(text)
                break  # If successful, exit the loop
            except Exception as e:
                retries += 1
                print(f"Error: {str(e)}. Retrying {retries}/{max_retries}...")  # Fix: Print retry information
                if retries == max_retries:
                    # Log the error and raise an exception to stop processing
                    print("Max retries exceeded. Logging the error and stopping the processing.")
                    raise
                time.sleep(5)

        number += 1
        limit_control_no += 1

        if predicted_label == 0:
            pos_num += 1
            score = 0
            sum_num += -1
        elif predicted_label == 1:
            neu_num += 1
            score = 1
            sum_num += 0
        elif predicted_label == 2:
            neg_num += 1
            score = 2
            sum_num += 1
        else:
            other_num += 1
            score = 4

        # Rate Limit Counterplan
        if limit_control_no == 200:
            limit_control_no = 0
            print('Wait for 10 seconds')
            time.sleep(10)

        append_list = [str(item['id']), str(item['author_id']), str(item['text']), str(score)]
        df_next = pd.DataFrame([append_list], columns=columns)
        df_output = pd.concat([df_output, df_next], ignore_index=True)

    sentiment_score = sum_num / number
    sentiment_index = 'Sentiment Index: ' + str(sentiment_score)
    tweets_number = 'Tweet_Number: ' + str(number)
    positive_number = str(pos_num)
    neutral_number = str(neu_num)
    negative_number = str(neg_num)
    file_name = 'File_Name: ' + str(tsv_files[h])
    print(file_name + ' ' + tweets_number + ' ' + str(sentiment_index))
    result_set = [str(tsv_files[h]), str(number), positive_number, neutral_number, negative_number, str(sentiment_score)]
    worksheet.append_row(result_set)

    # ☆
    directory = '/content/drive/My Drive/covid-twitter-usa-normal/result/gpt-3.5-turbo/los_angeles/3/' + str(tsv_files[h])
    df_output.to_csv(directory, sep='\t', index=False, header=False)


# Chicago

In [ ]:
import os
import gspread
import pandas as pd
from google.colab import auth

# Input Files
tsv_directory = '/content/drive/My Drive/covid-twitter-usa-normal/data/collection_data/chicago/3/1/'
tsv_files = os.listdir(tsv_directory)

# Sort the file names
tsv_files_sorted = sorted(tsv_files)

# Output File
url = "https://docs.google.com/spreadsheets/d/1RIQ8IZJk-Ooq-OWhO4Bi_AN_uEvlN6SC-QG6wWQKv5w/edit#gid=314682145"
sheet_name = '3'

# Output the contents of tsv_files
print("Contents of tsv_files:")
for file in tsv_files:
    print(file)

In [ ]:
import time
from openai import OpenAI

client = OpenAI()

number = 0
limit_control_no = 0

workbook = gc.open_by_url(url)
worksheet = workbook.worksheet(sheet_name)
header = ['file name', 'tweets', 'positive', 'neutral', 'negative', 'score']
worksheet.append_row(header)

# Function to classify sentiment using OpenAI API
def classify_sentiment(text):
    response = client.chat.completions.create(
        model="ft:gpt-3.5-turbo-1106:university-of-tsukuba::8RcZEQZm",
        messages=[
            {"role": "system", "content": "You are a sentiment classifier of tweets from citizens of a large US city collected during the COVID-19 pandemic. Classify into three values: 0 for positive, 1 for neutral, and 2 for negative. Positive sentiment includes: admire, amazing, assure, celebration, charm, eager, enthusiastic, excellent, fancy, fantastic, frolic, graceful, happy, joy, luck, majesty, mercy, nice, patience, perfect, proud, rejoice, relief, respect, satisfactorily, sensational, super, terrific, thank, vivid, wise, wonderful, zest, expectations and etc. Negative sentiment includes: abominable, anger, anxious, bad, catastrophe, cheap, complaint, condescending, deceit, defective, disappointment, embarrass, fake, fear, filthy, fool, guilt, hate, idiot, inflict, lazy, miserable, mourn, nervous, objection, pest, plot, reject, scream, silly, terrible, unfriendly, vile, wicked, and etc. Neutral sentiment: neither positive or negative, such as text without sentiment, stating a fact, question, news article, advertisement, solicitation, request, quote, unintelligible text, and etc. When the sentiment is mixed, such as both joy and sadness, use your judgment to choose the stronger emotion."},
            {"role": "user", "content": text}
        ],
        timeout=60  # Timeout set to 60 seconds
    )

    print("Prediction: " + response.choices[0].message.content)
    print("")
    try:
        return int(response.choices[0].message.content)
    except ValueError:
        print(f"Error: Unable to convert '{response.choices[0].message.content}' to int. Skipping this prediction.")
        return None

for h in range(len(tsv_files)):

    print(str(tsv_files[h]))

    columns = ['id', 'author_id', 'tweet', 'index']
    df_output = pd.DataFrame(columns=columns)

    sum_num = 0
    pos_num = 0
    neu_num = 0
    neg_num = 0
    other_num = 0
    score = 0
    number = 0
    limit_control_no = 0
    sentiment_score = 0
    sentiment_index = 0
    tweets_number = 0

    # ☆
    df_input = pd.read_table('/content/drive/My Drive/covid-twitter-usa-normal/data/collection_data/chicago/3/1/' + str(tsv_files[h]), names=('author_id', 'time', 'id', 'text', 'place'))
    print("The number of line:", len(df_input))
    # Duplicate Text Column
    df_input = df_input.drop_duplicates(subset='text')
    print("The number of line after duplication:", len(df_input))
    # Rate Limit Counterplan
    print('Wait for 3 seconds')
    time.sleep(3)

    for index, item in df_input.iterrows():

        print(index)
        text = item['text']
        print(text)

        time.sleep(0.5)

        max_retries = 3
        retries = 0

        while retries < max_retries:
            try:
                predicted_label = classify_sentiment(text)
                break  # If successful, exit the loop
            except Exception as e:
                retries += 1
                print(f"Error: {str(e)}. Retrying {retries}/{max_retries}...")  # Fix: Print retry information
                if retries == max_retries:
                    # Log the error and raise an exception to stop processing
                    print("Max retries exceeded. Logging the error and stopping the processing.")
                    raise
                time.sleep(5)

        number += 1
        limit_control_no += 1

        if predicted_label == 0:
            pos_num += 1
            score = 0
            sum_num += -1
        elif predicted_label == 1:
            neu_num += 1
            score = 1
            sum_num += 0
        elif predicted_label == 2:
            neg_num += 1
            score = 2
            sum_num += 1
        else:
            other_num += 1
            score = 4

        # Rate Limit Counterplan
        if limit_control_no == 200:
            limit_control_no = 0
            print('Wait for 10 seconds')
            time.sleep(10)

        append_list = [str(item['id']), str(item['author_id']), str(item['text']), str(score)]
        df_next = pd.DataFrame([append_list], columns=columns)
        df_output = pd.concat([df_output, df_next], ignore_index=True)

    sentiment_score = sum_num / number
    sentiment_index = 'Sentiment Index: ' + str(sentiment_score)
    tweets_number = 'Tweet_Number: ' + str(number)
    positive_number = str(pos_num)
    neutral_number = str(neu_num)
    negative_number = str(neg_num)
    file_name = 'File_Name: ' + str(tsv_files[h])
    print(file_name + ' ' + tweets_number + ' ' + str(sentiment_index))
    result_set = [str(tsv_files[h]), str(number), positive_number, neutral_number, negative_number, str(sentiment_score)]
    worksheet.append_row(result_set)

    # ☆
    directory = '/content/drive/My Drive/covid-twitter-usa-normal/result/gpt-3.5-turbo/chicago/3/' + str(tsv_files[h])
    df_output.to_csv(directory, sep='\t', index=False, header=False)